In [1]:
import numpy as np
import pandas as pd

In [2]:
train_j = pd.read_csv("/Users/miemie/Downloads/human_phone_train.csv")
test_j = pd.read_csv("/Users/miemie/Downloads/human_phone_test.csv")

In [6]:
train_j.values.shape #先列出所有数据的array形状

(3609, 563)

In [4]:
train_jy = train_j.values[...,1]

In [5]:
train_jy

array(['STANDING', 'STANDING', 'STANDING', ..., 'WALKING_UPSTAIRS',
       'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS'], dtype=object)

In [10]:
catelog = list(set(train_jy.tolist()))

In [11]:
catelog

['LAYING',
 'STANDING',
 'SITTING',
 'WALKING_DOWNSTAIRS',
 'WALKING_UPSTAIRS',
 'WALKING']

In [13]:
cate2int = dict((v,k) for k,v in enumerate(catelog))

In [14]:
cate2int

{'LAYING': 0,
 'STANDING': 1,
 'SITTING': 2,
 'WALKING_DOWNSTAIRS': 3,
 'WALKING_UPSTAIRS': 4,
 'WALKING': 5}

In [21]:
train_jy_arr = np.vectorize(lambda x: cate2int[x])(train_jy)

In [22]:
lambda x: cate2int[x]

<function __main__.<lambda>(x)>

In [24]:
a = lambda x: cate2int[x]

In [26]:
a("SITTING")

2

In [27]:
b = np.vectorize(a)

In [28]:
b(train_jy)

array([1, 1, 1, ..., 4, 4, 3])

In [29]:
train_jy_arr

array([1, 1, 1, ..., 4, 4, 3])

## Transform to One Hot Encoding

In [30]:
np.eye(len(catelog))

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [32]:
train_jy_hot = np.eye(len(catelog))[train_jy_arr]

In [33]:
train_jy_hot

array([[0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0.]])

In [34]:
train_x = train_j.values[...,2:]

In [35]:
train_x

array([[0.27899999999999997, -0.0196, -0.11, ..., -0.852, 0.182, -0.043],
       [0.27699999999999997, -0.0127, -0.10300000000000001, ..., -0.852,
        0.188, -0.0347],
       [0.27699999999999997, -0.0147, -0.107, ..., -0.85,
        0.18899999999999997, -0.0351],
       ...,
       [0.284, -0.00796, -0.11900000000000001, ..., -0.657, 0.272, 0.183],
       [0.207, 0.0246, -0.10400000000000001, ..., -0.657, 0.267, 0.188],
       [0.39299999999999996, -0.0178, -0.0902, ..., -0.807, 0.19,
        0.11800000000000001]], dtype=object)

In [36]:
train_x.shape

(3609, 561)

In [37]:
train_jy_hot.shape

(3609, 6)

## Build the model

In [38]:
from keras.layers import *
from keras.models import *
from keras.optimizers import Adam

#### pure linear

In [39]:
ipt = Input((561,))

In [40]:
x = Dense(len(catelog),activation="softmax")(ipt)
model = Model(ipt,x)

In [42]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 561)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3372      
Total params: 3,372
Trainable params: 3,372
Non-trainable params: 0
_________________________________________________________________


In [50]:
model.get_weights()[0].shape

(561, 6)

In [51]:
model.compile(optimizer="Adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [53]:
model.fit(train_x,train_jy_hot,batch_size=220,epochs=16,validation_split=.3,shuffle=True)

Train on 2526 samples, validate on 1083 samples
Epoch 1/16
2526/2526 [==============================] - 1s 265us/step - loss: 0.3898 - acc: 0.9125 - val_loss: 0.4302 - val_acc: 0.9021
Epoch 2/16
2526/2526 [==============================] - 1s 260us/step - loss: 0.3756 - acc: 0.9149 - val_loss: 0.4223 - val_acc: 0.8846
Epoch 3/16
2526/2526 [==============================] - 1s 259us/step - loss: 0.3612 - acc: 0.9200 - val_loss: 0.4008 - val_acc: 0.9021
Epoch 4/16
2526/2526 [==============================] - 1s 255us/step - loss: 0.3508 - acc: 0.9161 - val_loss: 0.3891 - val_acc: 0.9095
Epoch 5/16
2526/2526 [==============================] - 1s 260us/step - loss: 0.3379 - acc: 0.9188 - val_loss: 0.3809 - val_acc: 0.9215
Epoch 6/16
2526/2526 [==============================] - 1s 260us/step - loss: 0.3281 - acc: 0.9252 - val_loss: 0.3675 - val_acc: 0.9243
Epoch 7/16
2526/2526 [==============================] - 1s 257us/step - loss: 0.3167 - acc: 0.9291 - val_loss: 0.3625 - val_acc: 0.9197


### Neural network

In [54]:
ipt = Input((561,))

In [56]:
x = Dense(1000,activation="relu")(ipt)
x = Dense(len(catelog),activation="softmax")(x)
model = Model(ipt,x)

In [61]:
model.compile(optimizer=Adam(),loss="categorical_crossentropy",metrics=["accuracy"])
model.fit(train_x,train_jy_hot,batch_size=268,epochs=17,validation_split=.35,shuffle=True)

Train on 2345 samples, validate on 1264 samples
Epoch 1/17
2345/2345 [==============================] - 1s 487us/step - loss: 0.2179 - acc: 0.9288 - val_loss: 0.2195 - val_acc: 0.9296
Epoch 2/17
2345/2345 [==============================] - 1s 355us/step - loss: 0.0992 - acc: 0.9574 - val_loss: 0.2062 - val_acc: 0.9256
Epoch 3/17
2345/2345 [==============================] - 1s 360us/step - loss: 0.0668 - acc: 0.9736 - val_loss: 0.1462 - val_acc: 0.9581
Epoch 4/17
2345/2345 [==============================] - 1s 355us/step - loss: 0.0375 - acc: 0.9846 - val_loss: 0.1788 - val_acc: 0.9454
Epoch 5/17
2345/2345 [==============================] - 1s 358us/step - loss: 0.0324 - acc: 0.9919 - val_loss: 0.1480 - val_acc: 0.9565
Epoch 6/17
2345/2345 [==============================] - 1s 354us/step - loss: 0.0299 - acc: 0.9915 - val_loss: 0.1578 - val_acc: 0.9494
Epoch 7/17
2345/2345 [==============================] - 1s 356us/step - loss: 0.0315 - acc: 0.9898 - val_loss: 0.1544 - val_acc: 0.9494
